In [ ]:
!pip install datasets
#solution for importerror when running transforms repo scripts:
# https://discuss.huggingface.co/t/getting-error-while-fine-tuning-deberta-v3-large/11486 | https://colab.research.google.com/drive/1K_FGbcPG1hPltM-BYcx3IB10T-Hp2Lzy?usp=sharing#scrollTo=-FA1IjqHLADQ
!rm -r transformers
!git clone https://github.com/huggingface/transformers.git
!cd transformers
!pip install -q ./transformers

In [27]:
import torch
from PIL import Image
import os
from PIL import ImageFile
import json
import pathlib
from typing import Generator
import datasets
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


source repo for finetuning vlms: https://github.com/damian0815/finetune-clip-huggingface/tree/main

In [29]:
def collect_captioned_images(root_folder: str) -> Generator[tuple[str,str], None, None]:
    image_paths = []
    captions = []

    for directory, _, filenames in os.walk(root_folder):
        image_extensions = ['.jpg', '.jpeg']
        image_filenames = [f for f in filenames if os.path.splitext(f)[1] in image_extensions]
        for image_filename in image_filenames:
            caption_filename = os.path.splitext(image_filename)[0] + '.txt'
            caption_path = os.path.join(directory, caption_filename)
            if not os.path.exists(caption_path):
                continue

            with open(caption_path, 'r') as f:
                caption = f.read().replace('\n', ' ')

                image_path = os.path.join(directory, image_filename)
                yield image_path, caption


def convert_text_image_pairs_to_huggingface_json(root_folder, out_json):
    out_folder = os.path.dirname(root_folder)
    pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)
    with open(out_json, "w") as f:
        written_count = 0
        for image_path, caption in collect_captioned_images(root_folder):
            line_dict = {"image":image_path, "caption":caption}
            json_line = json.dumps(line_dict, indent=None, separators=(",",":"))
            #print(json_line)
            f.write(json_line + "\n")
            written_count += 1
        print(f"wrote {written_count} lines to {out_json}")

image-txt caption pair to HF compatiable json format

In [30]:
#convert image-caption pair from img-txt format to json format with image pathing
root_folder = '/content/drive/MyDrive/image-search-project/carBrandsSmallImgTextPairs'
out_json = '/content/drive/MyDrive/image-search-project/carBrandsSmallImgTextPairs.json'
convert_text_image_pairs_to_huggingface_json(root_folder, out_json)

wrote 93 lines to /content/drive/MyDrive/image-search-project/carBrandsSmallImgTextPairs.json


pull HF's transformers repo, finetuning code is under examples/pytorch/contrastive-image-txt/run_clip.py: https://github.com/huggingface/transformers/tree/main/examples/pytorch/contrastive-image-text

In [33]:
#fine tuning params:
repo_id =  "openai/clip-vit-base-patch32"
output_folder = "/content/drive/MyDrive/image-search-project/clip-finetune-results"
batch_size = 16
num_train_epochs =50

In [34]:
print(f"Finetuning {repo_id} for {num_train_epochs} epochs with batch size {batch_size}, and then saving output to {output_folder}.")
!python /content/transformers/examples/pytorch/contrastive-image-text/run_clip.py \
    --output_dir {output_folder} \
    --model_name_or_path {repo_id} \
    --train_file {out_json} \
    --image_column image \
    --overwrite_output_dir=True \
    --max_seq_length=77 \
    --num_train_epochs={num_train_epochs} \
    --remove_unused_columns=False \
    --do_train \
    --per_device_train_batch_size={batch_size} \
    --learning_rate="5e-5" --warmup_steps="0" --weight_decay 0.1
print("--\nDONE")
print(f"If it worked, trained data should be in {output_folder}")

Finetuning openai/clip-vit-base-patch32 for 50 epochs with batch size 16, and then saving output to /content/drive/MyDrive/image-search-project/clip-finetune-results.
2025-05-02 07:06:56.848205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746169616.881896   10637 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746169616.892542   10637 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
05/02/2025 07:07:01 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
05/02/2025 07:07:01 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batc

In [41]:
#testing finetuned clip model, load the output_dir
from transformers import CLIPProcessor, CLIPModel
clip_model = CLIPModel.from_pretrained("/content/drive/MyDrive/image-search-project/clip-finetune-results").to(device)